# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
import random
import torch
import numpy as np
import time
import pickle
from itertools import count
from collections import deque
import os
import matplotlib.pyplot as plt
%matplotlib inline

from buffer import ReplayBuffer
from maddpg import MADDPG
from utilities import transpose_list, transpose_to_tensor
#  conda install -c conda-forge tensorboardx  -- cause troubles with protobuf version
# rather pip install tensorboardX
from tensorboardX import SummaryWriter

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [ ]:
env = UnityEnvironment(file_name="../Tennis_Windows_x86_64/Tennis.exe")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents = 2
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action = 2
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space -- Each observes a state with length: 24
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

In [ ]:
# env.close()

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [ ]:
random_actions = False

In [ ]:
if random_actions:
    for i in range(1, 6):                                      # play game for 5 episodes
        env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
        states = env_info.vector_observations                  # get the current state (for each agent)
        scores = np.zeros(num_agents)                          # initialize the score (for each agent)
        while True:
            actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
            print(actions)  # [[ 0.3190391  -0.24937038], [ 1.46210794 -2.06014071]]
            actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
            env_info = env.step(actions)[brain_name]           # send all actions to tne environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            scores += env_info.rewards                         # update the score (for each agent)
            states = next_states                               # roll over states to next time step
            if np.any(dones):                                  # exit loop if episode finished
                break
        print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

When finished, you can close the environment.

In [ ]:
if random_actions:
    env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
def seeding(seed=1):
    np.random.seed(seed)
    torch.manual_seed(seed)

In [ ]:
# configuration and hyper-parameters
seed_id = 1
seeding(seed_id)
n_agents = 2              # fix by the env
parallel_envs = 1         # cannot do anything but one
number_of_episodes = 100
episode_length = 80       # number of steps to stop an episode 
batchsize = 2
save_interval = 10        # how many episodes to save policy
episode_per_update = 2 * parallel_envs  # how many episodes before update
buffer_size = 5000        # how many episodes kept as worth of replay
buffer = ReplayBuffer(int(buffer_size*episode_length))

# amplitude of OU noise -- slowly decrease noise to 0
noise = 2
mini_noise = 0.01
noise_reduction = 0.9999

In [ ]:
# tensorboard logging
log_path = os.getcwd()+"/log"
model_dir= os.getcwd()+"/model_dir"
os.makedirs(model_dir, exist_ok=True)
logger = SummaryWriter(log_dir=log_path)
agent0_reward = []
agent1_reward = []

# initialize policy and critic
maddpg = MADDPG()

# performance measurement
threshold_success = 0.5
scores_global = []                # collect scores for each episode
scores_deque = deque(maxlen=100)  # collect 100 latest scores
start_learning = time.time()
stop = False

# show progressbar
import progressbar as pb
widget = ['episode: ', pb.Counter(),'/',str(number_of_episodes),' ', 
          pb.Percentage(), ' ', pb.ETA(), ' ', pb.Bar(marker=pb.RotatingMarker()), ' ' ]
timer = pb.ProgressBar(widgets=widget, maxval=number_of_episodes).start()

In [ ]:
for episode in range(0, number_of_episodes, parallel_envs):
    
    # book-keeping
    timer.update(episode)
    start_episode = time.time()
    reward_this_episode = np.zeros((parallel_envs, n_agents))

    # reset the environment
    env_info = env.reset(train_mode=False)[brain_name]
    states = env_info.vector_observations
    obs = [states]
    # !!! ToDo: define obs_full !!!! 
    obs_full = [np.concatenate(states, axis=None)]
    # obs_full = [states[0]]

    # save info or not
    save_info = ((episode) % save_interval < parallel_envs or episode==number_of_episodes-parallel_envs)

    for _ in range(episode_length):
        # action input needs to be transposed
        actions = maddpg.act(transpose_to_tensor(obs), noise=max(noise, mini_noise))
        noise *= noise_reduction
#         actions_array = torch.stack(actions).detach().numpy()
        actions_array = torch.stack(actions).cpu().detach().numpy()
        actions_for_env = np.rollaxis(actions_array,1)
        actions = actions_for_env[0]

        # step forward one frame
        # next_obs, next_obs_full, rewards, dones, info = env.step(actions_for_env)
        env_info = env.step(actions)[brain_name]      # send all actions to the environment
        next_states = env_info.vector_observations    # get next state (for each agent)
        rewards = env_info.rewards                    # get reward (for each agent)
        dones = env_info.local_done                   # see if episode finished
        
        # adapt format
        # !!! ToDo: define next_obs_full !!!! 
        # next_obs_full = [next_states[0]]
        next_obs_full = [np.concatenate(next_states, axis=None)]
#         print("next_states = {}".format(next_states))
#         print("next_obs_full = {}".format(next_obs_full))
        next_obs = [next_states]
        dones = [dones]
        rewards = [rewards]

        # add data to buffer
        transition = (obs, obs_full, actions_for_env, rewards, next_obs, next_obs_full, dones)
        buffer.push(transition)

        # before transitioning to new step
        reward_this_episode += rewards
        obs = next_obs

    # --- episode is over ---

    # update models once after every episode_per_update
    if len(buffer) > batchsize and episode % episode_per_update < parallel_envs:
        print("epsiode = {} -- time for update with batchsize = {}".format(episode, batchsize))
        for a_i in range(n_agents):
            samples = buffer.sample(batchsize)
            maddpg.update(samples, a_i, logger)
        maddpg.update_targets() #  soft update the target network towards the actual networks

    # measure performance
    score = max(reward_this_episode[0])
    scores_deque.append(score)
    scores_global.append(score)
    print('{}-th episode, score_ep= {:.4f}, avg_score= {:.4f}, duration[ep]= {:.2f} [s]'\
          .format(episode, score, np.mean(scores_deque), time.time()-start_episode), end="\n")
    for i in range(parallel_envs):
        agent0_reward.append(reward_this_episode[i,0])
        agent1_reward.append(reward_this_episode[i,1])
    
    # log performance
    logger.add_scalar('score', score, episode)          # not working??
    logger.add_scalar('avg_score', np.mean(scores_deque), episode)  # not working??
    if episode % 3 == 0 or episode == number_of_episodes-1:
        avg_rewards = [np.mean(agent0_reward), np.mean(agent1_reward)]
        agent0_reward = []
        agent1_reward = []
        for a_i, avg_rew in enumerate(avg_rewards):
            logger.add_scalar('agent%i/mean_episode_rewards' % a_i, avg_rew, episode)

    # Test if success -- Instructions:
    # After each episode, we add up the rewards that each agent received (without discounting)
    # to get a score for each agent.
    # This yields 2 (potentially different) scores.
    # We then take the maximum of these 2 scores.
    # This yields a single score for each episode.
    # The environment is considered solved,
    # when the average (over 100 episodes) of those scores is at least +0.5
    if np.mean(scores_deque) > threshold_success:
        print("SUCCESS -- after {} episodes and {:.2f} min".format(episode, (time.time()-start_learning)/60))
        save_info = True
        stop = True

    #saving model
    save_dict_list =[]
    if save_info:
        for i in range(n_agents):
            save_dict = {'actor_params' : maddpg.maddpg_agent[i].actor.state_dict(),
                         'actor_optim_params': maddpg.maddpg_agent[i].actor_optimizer.state_dict(),
                         'critic_params' : maddpg.maddpg_agent[i].critic.state_dict(),
                         'critic_optim_params' : maddpg.maddpg_agent[i].critic_optimizer.state_dict()}
            save_dict_list.append(save_dict)
            torch.save(save_dict_list, os.path.join(
                model_dir, 'episode-{}.pt'.format(episode)))

    if stop:
        break

env.close()
logger.close()
timer.finish()

In [ ]:
env.close()